In [1]:
import json
import collections
import numpy as np

In [2]:
with open('../json/formulas.json', 'r') as f:
    formulas = json.load(f)
with open('../json/constants.json', 'r') as f:
    constants = json.load(f)
with open('../json/input_variables.json', 'r') as f:
    input_variables = json.load(f)

In [3]:
# important variables
roots = ['NBPT', 'REVKIRE', 'BCSG', 'BRDS', 'IBM23', 'TXMOYIMP', 'NAPTIR', 'IINET', 'RRRBG', 'RNI', 'IDRS3', 'IAVIM']

# Get dependencies and find a good computation order

## Get dependencies

In [4]:
def get_children(node):
    nodetype = node['nodetype']

    if nodetype == 'symbol':
        name = node['name']
        return set([name])
        
    elif nodetype == 'float':
        return set()
    
    elif nodetype == 'call':
        args = node['args']
        children = set()
        for arg in args:
            children = children | get_children(arg)

        return children
        
    raise ValueError('Unknown type : %s'%nodetype)

In [5]:
children_dict = {}
for name, formula in formulas.items():
    children_dict[name] = get_children(formula)

## Find dependencies without formula

In [6]:
inputs_list = []
for e in input_variables:
    inputs_list.append(e['alias'])
    inputs_list.append(e['name'])

In [7]:
unknown_variables = set()
for k, v in children_dict.items():
    for var in v:
        if (var not in formulas) and (var not in constants) and (var not in inputs_list):
            unknown_variables.add(var)

In [8]:
len(unknown_variables)

1435

## Parents

In [9]:
parents_dict = {}

for k in children_dict:
    parents_dict[k] = set()

for parent, children in children_dict.items():
    for child in children:
        if child in children_dict:
            parents_dict[child].add(parent)

## Dependencies

In [10]:
to_inspect = roots[:]
dependencies_formulas = []
dependencies_constants = []
dependencies_inputs = []
dependencies_unknown = []

while to_inspect:
    node = to_inspect.pop()
    
    if node in dependencies_formulas:
        continue
     
    for child in children_dict[node]:
        if child in formulas:
            if (child not in dependencies_formulas) and (child not in to_inspect):
                to_inspect.append(child)
        elif child in constants:
            if child not in dependencies_constants:
                dependencies_constants.append(child)
        elif child in inputs_list:
            if child not in dependencies_inputs:
                dependencies_inputs.append(child)
        elif child in unknown_variables:
            if child not in dependencies_unknown:
                dependencies_unknown.append(child)
        else:
            raise Exception('Unknown variable category : %s for parent %s.'%(child, node))

    dependencies_formulas.append(node)


In [11]:
len(dependencies_formulas)

4740

In [12]:
len(dependencies_constants)

233

In [13]:
len(dependencies_inputs)

1463

In [14]:
len(dependencies_unknown)

782

## Ignore useless variables

In [15]:
formulas_light = {k: formulas[k] for k in dependencies_formulas}
constants_light = {k: constants[k] for k in dependencies_constants}
inputs_light = dependencies_inputs
unknowns_light = dependencies_unknown

## Find cycles

### Adjacency matrix : too long (do not execute the following cells)

In [ ]:
index = {}
reverse_index = []
for i, k in enumerate(formulas_light):
    index[k] = i
    reverse_index.append(k)

In [ ]:
n = len(index)
adj = np.zeros((n, n))

In [ ]:
for parent in formulas_light:
    children = children_dict[parent]
    for child in children:
        if child in children_dict:
            adj[index[parent], index[child]] = 1

In [ ]:
current_power = 1
while current_power <= n:
    print(current_power)
    adj = np.dot(adj, adj)
    current_power += 1

In [ ]:
np.diag(adj).sum()

### Graph traversal : too long

In [16]:
children_light = {}
for formula in formulas_light:
    children_light[formula] = []
    for child in children_dict[formula]:
        if child in formulas_light:
            children_light[formula].append(child)

In [ ]:
def cycle_research(node, genealogy):
    if node in genealogy:
        print('Cycle detected : %s'%str(genealogy))
    else:
        for child in children_light[node]:
            child_genealogy = list(genealogy)
            child_genealogy.append(node)
            cycle_research(child, child_genealogy)
        

In [ ]:
# do not execute this cell (too long)
cycle_research('IRN', [])

### Tarjan's strongly connected components algorithm

In [17]:
class State():
    def __init__(self):
        self.current_index = 0
        self.stack = []
        self.strongly_connected_components = []
        self.tarjan_indexes = {}
        self.low_link = {}
s = State()

In [18]:
def strong_connect(node, s):
    s.tarjan_indexes[node] = s.current_index
    s.low_link[node] = s.current_index
    s.current_index += 1
    s.stack.append(node)

    for child in children_light[node]:
        if child not in s.tarjan_indexes:
            strong_connect(child, s)
            s.low_link[node] = min(s.low_link[node], s.low_link[child])
        elif child in s.stack:
            s.low_link[node] = min(s.low_link[node], s.tarjan_indexes[child])

    if s.low_link[node] == s.tarjan_indexes[node]:
        new_component = []
        while s.stack[-1] != node:
            new_component.append(s.stack.pop())
        new_component.append(s.stack.pop())
        s.strongly_connected_components.append(new_component)

In [19]:
for node in formulas_light:
    if node not in s.tarjan_indexes:
        strong_connect(node, s)

In [20]:
component_size = [len(comp) for comp in s.strongly_connected_components]
c = collections.Counter(component_size)
c

Counter({1: 4740})

## Find good computing order

In [21]:
def find_order(node):
    is_leaf = True
    for child in children_light[node]:
        if child not in computing_order:
            find_order(child)
            
    computing_order.append(node)


In [22]:
computing_order = []
for root in roots:
    find_order(root)

In [23]:
with open('../json/computing_order.json', 'w') as f:
    f.write(json.dumps(computing_order))

In [24]:
with open('../json/children_light.json', 'w') as f:
    f.write(json.dumps(children_light))
    
with open('../json/formulas_light.json', 'w') as f:
    f.write(json.dumps(formulas_light))
    
with open('../json/constants_light.json', 'w') as f:
    f.write(json.dumps(constants_light))
    
with open('../json/inputs_light.json', 'w') as f:
    f.write(json.dumps(inputs_light))
    
with open('../json/unknowns_light.json', 'w') as f:
    f.write(json.dumps(unknowns_light))    

## Tree depth

In [ ]:
# recursive : too long (do not execute)
def compute_depth(node):
    children_depth = [compute_depth(child) for child in children_light[node]]
    if children_depth:
        return 1 + max(children_depth)
    else:
        return 1

In [25]:
depths = {}
for var in computing_order:
    children_depth = [depths[child] for child in children_light[var]]
    if children_depth:
        depths[var] = 1 + max(children_depth)
    else:
        depths[var] = 1

In [26]:
depths['IINET']

1119